In [1]:
# 생성자 모델 : 가상의 이미지를 판별하는 모델

# 판별자 모델 : 이미지가 진짜인지 가짜인지 판별하는 모델..

# gan 모델 : 생성자와 판별자가 결합된 모델..
          #  생성자가 생성한 이미지를 판별자에게 판별하도록 하는 모델
          #  생성자가 생성된 이미지가 True라벨로 학습 -> 생성자가 진짜 이미지와 같은 이미지를 생성하도록 학습..

In [4]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import os
if not os.path.exists('./gan_images'):
  os.mkdir('./gan_images')

In [9]:
128*7*7

6272

In [23]:
# 생성자

generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7,7,128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
generator.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 6272)              633472    
                                                                 
 batch_normalization_10 (Bat  (None, 6272)             25088     
 chNormalization)                                                
                                                                 
 reshape_4 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_5 (UpSampling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_11 (Bat  (None, 14, 14, 64)     

In [16]:
# 판별자

discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation_2 (Activation)   (None, 14, 14, 64)        0         
                                                                 
 dropout_2 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_3 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_3 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 6272)             

In [24]:
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)

In [26]:
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_14 (Sequential)  (None, 28, 28, 1)         865281    
                                                                 
 sequential_8 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 1,065,474
Non-trainable params: 12,672
_________________________________________________________________


In [27]:
def gan_train(epoch, batch_size, saving_inerval):
  (x_train, _), (_, _) = mnist.load_data()
  x_train = x_train.reshape(x_train.shape[0], 28, 28, 1),astype('float32')
  x_train = (x_train - 127.5) / 127.5

  true = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for i in range(epoch):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train
    d_loss_real = disriminator.train_on_batch(imgs, true)

    noise = np.random.normal(0,1, (batch_size, 100))
    get_imgs = generator.prediction(noise)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
 
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    d_loss = gan.train_on_batch(noise, true)

    print(i, d_loss, g_loss)

    if i % saving_interval == 0:
      noise = np.random.normal(0,1,(25,100))
      gan_imgs = ganarator.predict(noise)

      gen_imgs = 0.5 * gen_imgs * 0.5

      fig. axs = plt.subplots(5,5)
      count = 0

      for i in range(5):
        for k in range(5):
          axs[j, k].imshow(gem_imgs[count, :, :, 0], cmap='gray')
          axs[j, k].axis['off']
          count += 1